In [ ]:
%load_ext autoreload
%autoreload 2

import meshio
import numpy as np
import torch

from fem import eval, fea
from graphs import build_graph
from utils import msh_to_trimesh, visualize

# Example Graph Generation

In [ ]:
stl = "meshes/cantilever.stl"
msh = "meshes/cantilever.msh"
mesh = meshio.read(msh)


f = np.array([[10.0, 0.0, 0.0], [5.0, 0.0, 0.0]])
points = np.array([[0.0, 9.0, 30.0], [0.0, 9.0, 20.0]])
contacts = list(zip(points, f))

domain, stresses_vm = fea(
    contacts, contact_radius=2.0, debug=False, filename_stl=stl, filename_msh=msh
)
y = eval(domain, stresses_vm, mesh.points)
graph = build_graph(mesh, y, contacts)
print("Graph is valid?", graph.validate(raise_on_error=True))

In [ ]:
x1 = torch.tensor([[-10, 1, -30], [-10, 1, 20], [-10, -9, -30], [0, 1, -30]])
x2 = torch.tensor([[-10, 1, -20], [-10, 1, 30], [-10, -9, -20], [0, 1, -20]])
f1 = torch.tensor([[10, 0, 0], [10, 0, 0], [10, 0, 0], [10, 0, 0]])
f2 = torch.tensor([[5, 0, 0], [5, 0, 0], [5, 0, 0], [5, 0, 0]])
X = [x1, x2]
Y = [f1, f2]

inter = torch.stack([torch.stack(X), torch.stack(Y)], dim=1).reshape(-1, 4, 3)
attrs = inter.permute(1, 0, 2).reshape(4, -1)

In [ ]:
torch.stack([torch.stack(X), torch.stack(Y)], dim=1)

## Visualize Graphs

In [ ]:
mesh = meshio.read(msh)
mesh = msh_to_trimesh(mesh)
visualize(mesh, graph, jupyter_backend="html", force_arrows=True)

## Load Graphs and Visualize

In [ ]:
data = torch.load("data/abc_1_1.pt", weights_only=False)
graphs = data["graphs"]
meshes = data["meshes"]
idx = 1
print(f"Visualizing graph {idx} from mesh {meshes[idx]}...")
mesh = meshio.read(f"meshes/abc/msh/{meshes[idx]}.msh")
mesh = msh_to_trimesh(mesh)

visualize(mesh, graphs[idx], v_idx=2, jupyter_backend="html", force_arrows=True)

In [ ]:
p1 = torch.tensor([[1.0, 2.0, 3.0], [0.0, 1.0, 2.0]])
p2 = torch.tensor([[4.0, 5.0, 6.0], [1.0, 2.0, 3.0]])
p = [p1, p2]

f1 = torch.tensor([[0.1, 0.2, 0.3], [0.0, 0.1, 0.2]])
f2 = torch.tensor([[0.4, 0.5, 0.6], [0.1, 0.2, 0.3]])
f = [f1, f2]

torch.stack([torch.stack(p), torch.stack(f)], dim=2)

# Evenly-Distributed-Load Dataset Generation

In [ ]:
def make_dataset(
    mesh: meshio.Mesh,
    f: np.ndarray,
    points: np.ndarray,
    stl="cantilever.stl",
    msh="cantilever.msh",
):
    """Generate n graphs from given contact points on one given mesh."""
    graphs = []
    for p, f_vec in zip(points, f):
        contacts = [(p, f_vec)]

        domain, stresses_vm = fea(
            contacts,
            contact_radius=2.0,
            debug=False,
            filename_stl=stl,
            filename_msh=msh,
        )
        y = eval(domain, stresses_vm, mesh.points)
        graphs.append(build_graph(mesh, y, contacts))

    return graphs


stl = "meshes/cantilever.stl"
msh = "meshes/cantilever.msh"
mesh = meshio.read(msh)

points = np.array(
    [
        [0.0, 5.0, 10.0],
        [0.0, 5.0, 20.0],
        [0.0, 5.0, 30.0],
        [0.0, 5.0, 40.0],
    ]
)
f = np.tile([10.0, 0.0, 0.0], (len(points), 1))

graphs = make_dataset(mesh, f, points, stl=stl, msh=msh)
torch.save(graphs, "cantilever_1_4.pt")